## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Oct. 27th Nov. 3rd


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-31-2024,Condo/Co-op,704 Flanders O,Delray Beach,FL,33484.0,65000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/704-Fla...,Beaches MLS,RX-11033054,N,Y,26.440095,-80.143513


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
60,PAST SALE,2024-11-04,Condo/Co-op,434 Saxony J,Delray Beach,FL,33446.0,84000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/434-Sax...,Beaches MLS,RX-11009231,N,Y,26.440263,-80.150386
2,PAST SALE,2024-11-03,Condo/Co-op,211 Wellington L,West Palm Beach,FL,33417.0,150000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/211-...,Beaches MLS,RX-11033597,N,Y,26.709470,-80.132007
46,PAST SALE,2024-11-01,Condo/Co-op,1495 Lake Crystal Dr Unit B,West Palm Beach,FL,33411.0,230000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/1495...,Beaches MLS,RX-11015078,N,Y,26.700518,-80.170581
91,PAST SALE,2024-11-01,Condo/Co-op,303 NE 17th Ave #204,Boynton Beach,FL,33435.0,105000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/303-NE...,Beaches MLS,RX-10981893,N,Y,26.544997,-80.062136
64,PAST SALE,2024-11-01,Condo/Co-op,2871 N Ocean Blvd Unit D202,Boca Raton,FL,33431.0,897000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2871-N-Oc...,Beaches MLS,RX-10994200,N,Y,26.376680,-80.068894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,PAST SALE,2024-10-28,Condo/Co-op,6028 Forest Hl #205,West Palm Beach,FL,33415.0,148000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/6028...,Beaches MLS,RX-10970600,N,Y,26.652105,-80.137049
75,PAST SALE,2024-10-28,Condo/Co-op,70 Eastgate Dr Unit C,Boynton Beach,FL,33436.0,225000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/70-Eas...,Beaches MLS,RX-10986646,N,Y,26.487921,-80.096275
42,PAST SALE,2024-10-28,Condo/Co-op,547 Saxony Unit L,Delray Beach,FL,33446.0,90000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/547-Sax...,Beaches MLS,RX-11015083,N,Y,26.440263,-80.150386
80,PAST SALE,2024-10-28,Condo/Co-op,2012 Guildford A,Boca Raton,FL,33434.0,125000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2012-Guil...,Beaches MLS,RX-10992990,N,Y,26.386810,-80.175410


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

114


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$440,082


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$294


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$50,169,300


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach
Input Previous Week Condo Sales Total (number units sold):
80
Input Previous Week Condo Average Sales Price:
347880
Input Previous Week Condo Average PSF:
256
Input Previous Week Condo Sales total (ex: 198_000_000)
27_800_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-28,Condo/Co-op,200 E Palmetto Park Rd Ph -11,Boca Raton,FL,33432.0,3990000.0,3.0,3.5,200 East Palmetto Park Condo,4154.0,180.0,2009.0,NaN,961.0,4167.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-10918738,N,Y,26.350338,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-28,Condo/Co-op,200 E Palmetto Park Rd Ph -11,Boca Raton,FL,33432.0,3990000.0,3.0,3.5,200 East Palmetto Park Condo,4154.0,180.0,2009.0,NaN,961.0,4167.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-10918738,N,Y,26.350338,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange
1,PAST SALE,2024-10-28,Condo/Co-op,3211 S Ocean Blvd #802,Highland Beach,FL,33487.0,3050000.0,3.0,4.0,Villa Mare Condo,2657.0,NaN,1991.0,NaN,1148.0,3313.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3211-...,Beaches MLS,RX-11008108,N,Y,26.417668,-80.064013,https://www.redfin.com/FL/Highland-Beach/3211-...,Geoffrey Braboy,Compass Florida LLC,None,None,Geoffrey Braboy,Compass Florida LLC,None,None,2,blue
2,PAST SALE,2024-11-01,Condo/Co-op,200 SE Mizner Blvd #711,Boca Raton,FL,33432.0,2750000.0,3.0,3.0,Alina Boca Raton Condo,2035.0,NaN,2021.0,NaN,1351.0,2663.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11018049,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Edward Pitlake,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Shaina MacMahon,Compass Florida LLC,None,None,3,blue
3,PAST SALE,2024-10-31,Condo/Co-op,3550 S Ocean Blvd Unit 2b,South Palm Beach,FL,33480.0,2625000.0,3.0,3.5,3550 South Ocean Condo,2982.0,NaN,2019.0,NaN,880.0,3558.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Beaches MLS,RX-10863520,N,Y,26.611901,-80.038420,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Patricia Towle,"Sotheby's Intl. Realty, Inc.",None,None,Brie Segota,Coffee Cake And Real Estate,None,None,4,blue
4,PAST SALE,2024-10-30,Condo/Co-op,200 SE Mizner Blvd #608,Boca Raton,FL,33432.0,2150000.0,2.0,2.5,Alina Boca Raton Condo,1950.0,381815.0,2021.0,NaN,1103.0,2596.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-10986420,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,None,None,Rochelle LeCavalier,Douglas Elliman,None,None,5,blue
5,PAST SALE,2024-10-31,Condo/Co-op,350 S Ocean Blvd Ph -C,Boca Raton,FL,33432.0,1900000.0,3.0,3.0,Beresford,2296.0,NaN,1977.0,NaN,828.0,3894.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10978366,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Mona Pincus,Stonegate Realty Corporation,None,None,6,blue
6,PAST SALE,2024-10-30,Condo/Co-op,3594 S Ocean Blvd #1103,Highland Beach,FL,33487.0,1250000.0,2.0,2.0,Highland Beach Club,1400.0,NaN,1997.0,NaN,893.0,1469.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3594-...,Beaches MLS,F10458939,N,Y,26.411722,-80.065580,https://www.redfin.com/FL/Highland-Beach/3594-...,Tanya Nilsson,The Network Real Estate Broker,None,None,Simon Isaacs,Simon Isaacs Real Estate LLC,None,None,7,blue
7,PAST SALE,2024-10-31,Condo/Co-op,701 S Olive Ave #1204,West Palm Beach,FL,33401.0,1225000.0,2.0,2.5,Two City Plaza Condo,1625.0,NaN,2008.0,NaN,754.0,1562.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/701-...,Beaches MLS,RX-11033422,N,Y,26.706646,-80.052576,https://www.redfin.com/FL/West-Palm-Beach/701-...,Christopher Cronican,Douglas Elliman (Palm Beach),Lori Cronican,Douglas Elliman (Palm Beach),Jenna Steffens,Douglas Elliman (Palm Beach),None,None,8,blue
8,PAST SALE,2024-10-31,Condo/Co-op,1701 S Flagler Dr #1102,West 

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-28,Condo/Co-op,200 E Palmetto Park Rd Ph -11,Boca Raton,FL,33432.0,3990000.0,3.0,3.5,200 East Palmetto Park Condo,4154.0,180.0,2009.0,NaN,961.0,4167.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-10918738,N,Y,26.350338,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_11042024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'200 East Palmetto Park Condo at 200 E Palmetto Park Rd Ph -11 in Boca Raton'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-28,Condo/Co-op,200 E Palmetto Park Rd Ph -11,Boca Raton,FL,33432.0,3990000.0,3.0,3.5,200 East Palmetto Park Condo,4154.0,180.0,2009.0,NaN,961.0,4167.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-10918738,N,Y,26.350338,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange,200 East Palmetto Park Condo at 200 E Palmetto...
1,PAST SALE,2024-10-28,Condo/Co-op,3211 S Ocean Blvd #802,Highland Beach,FL,33487.0,3050000.0,3.0,4.0,Villa Mare Condo,2657.0,NaN,1991.0,NaN,1148.0,3313.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3211-...,Beaches MLS,RX-11008108,N,Y,26.417668,-80.064013,https://www.redfin.com/FL/Highland-Beach/3211-...,Geoffrey Braboy,Compass Florida LLC,None,None,Geoffrey Braboy,Compass Florida LLC,None,None,2,blue,Villa Mare Condo at 3211 S Ocean Blvd #802 in ...
2,PAST SALE,2024-11-01,Condo/Co-op,200 SE Mizner Blvd #711,Boca Raton,FL,33432.0,2750000.0,3.0,3.0,Alina Boca Raton Condo,2035.0,NaN,2021.0,NaN,1351.0,2663.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11018049,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Edward Pitlake,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Shaina MacMahon,Compass Florida LLC,None,None,3,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
3,PAST SALE,2024-10-31,Condo/Co-op,3550 S Ocean Blvd Unit 2b,South Palm Beach,FL,33480.0,2625000.0,3.0,3.5,3550 South Ocean Condo,2982.0,NaN,2019.0,NaN,880.0,3558.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Beaches MLS,RX-10863520,N,Y,26.611901,-80.038420,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Patricia Towle,"Sotheby's Intl. Realty, Inc.",None,None,Brie Segota,Coffee Cake And Real Estate,None,None,4,blue,3550 South Ocean Condo at 3550 S Ocean Blvd Un...
4,PAST SALE,2024-10-30,Condo/Co-op,200 SE Mizner Blvd #608,Boca Raton,FL,33432.0,2150000.0,2.0,2.5,Alina Boca Raton Condo,1950.0,381815.0,2021.0,NaN,1103.0,2596.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-10986420,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,None,None,Rochelle LeCavalier,Douglas Elliman,None,None,5,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
5,PAST SALE,2024-10-31,Condo/Co-op,350 S Ocean Blvd Ph -C,Boca Raton,FL,33432.0,1900000.0,3.0,3.0,Beresford,2296.0,NaN,1977.0,NaN,828.0,3894.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10978366,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Mona Pincus,Stonegate Realty Corporation,None,None,6,blue,Beresford at 350 S Ocean Blvd Ph -C in Boca Raton
6,PAST SALE,2024-10-30,Condo/Co-op,3594 S Ocean Blvd #1103,Highland Beach,FL,33487.0,1250000.0,2.0,2.0,Highland Beach Club,1400.0,NaN,1997.0,NaN,893.0,1469.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3594-...,Beaches MLS,F10458939,N,Y,26.411722,-80.065580,https://www.redfin.com/FL/Highland-Beach/3594-...,Tanya Nilsson,The Network Real Estate Broker,None,None,Simon Isaacs,Simon Isaacs Real Estate LLC,None,None,7,blue,Highland Beach Club at 3594 S Ocean Blvd #1103...
7,PAST SALE,2024-10-31,Condo/Co-op,701 S Olive Ave #1204,West Palm Beach,FL,33401.0,1225000.0,2.0,2.5,Two City Plaza Condo,1625.0,NaN,2008.0,NaN,754.0,1562.0,Sold,NaN,NaN,https:/

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 200 East Palmetto Park Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,100,000M to $3,990,000M
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Highland Beach, South Palm Beach, Boca Raton, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 114 condo sales totaling $50,169,300 million from Oct. 27th to Nov. 3rd. The previous week, brokers closed 80 condo sales totaling $27,800,000.

Last week’s units sold for an average of $440,082, higher than the $347,880 average price from the previous week. The a

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-28,Condo/Co-op,200 E Palmetto Park Rd Ph -11,Boca Raton,FL,33432.0,3990000.0,3.0,3.5,200 East Palmetto Park Condo,4154.0,180.0,2009.0,NaN,961.0,4167.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-10918738,N,Y,26.350338,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange,200 East Palmetto Park Condo at 200 E Palmetto...
1,PAST SALE,2024-10-28,Condo/Co-op,3211 S Ocean Blvd #802,Highland Beach,FL,33487.0,3050000.0,3.0,4.0,Villa Mare Condo,2657.0,NaN,1991.0,NaN,1148.0,3313.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3211-...,Beaches MLS,RX-11008108,N,Y,26.417668,-80.064013,https://www.redfin.com/FL/Highland-Beach/3211-...,Geoffrey Braboy,Compass Florida LLC,None,None,Geoffrey Braboy,Compass Florida LLC,None,None,2,blue,Villa Mare Condo at 3211 S Ocean Blvd #802 in ...
2,PAST SALE,2024-11-01,Condo/Co-op,200 SE Mizner Blvd #711,Boca Raton,FL,33432.0,2750000.0,3.0,3.0,Alina Boca Raton Condo,2035.0,NaN,2021.0,NaN,1351.0,2663.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11018049,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Edward Pitlake,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Shaina MacMahon,Compass Florida LLC,None,None,3,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
3,PAST SALE,2024-10-31,Condo/Co-op,3550 S Ocean Blvd Unit 2b,South Palm Beach,FL,33480.0,2625000.0,3.0,3.5,3550 South Ocean Condo,2982.0,NaN,2019.0,NaN,880.0,3558.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Beaches MLS,RX-10863520,N,Y,26.611901,-80.038420,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Patricia Towle,"Sotheby's Intl. Realty, Inc.",None,None,Brie Segota,Coffee Cake And Real Estate,None,None,4,blue,3550 South Ocean Condo at 3550 S Ocean Blvd Un...
4,PAST SALE,2024-10-30,Condo/Co-op,200 SE Mizner Blvd #608,Boca Raton,FL,33432.0,2150000.0,2.0,2.5,Alina Boca Raton Condo,1950.0,381815.0,2021.0,NaN,1103.0,2596.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-10986420,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,None,None,Rochelle LeCavalier,Douglas Elliman,None,None,5,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
5,PAST SALE,2024-10-31,Condo/Co-op,350 S Ocean Blvd Ph -C,Boca Raton,FL,33432.0,1900000.0,3.0,3.0,Beresford,2296.0,NaN,1977.0,NaN,828.0,3894.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10978366,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Mona Pincus,Stonegate Realty Corporation,None,None,6,blue,Beresford at 350 S Ocean Blvd Ph -C in Boca Raton
6,PAST SALE,2024-10-30,Condo/Co-op,3594 S Ocean Blvd #1103,Highland Beach,FL,33487.0,1250000.0,2.0,2.0,Highland Beach Club,1400.0,NaN,1997.0,NaN,893.0,1469.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3594-...,Beaches MLS,F10458939,N,Y,26.411722,-80.065580,https://www.redfin.com/FL/Highland-Beach/3594-...,Tanya Nilsson,The Network Real Estate Broker,None,None,Simon Isaacs,Simon Isaacs Real Estate LLC,None,None,7,blue,Highland Beach Club at 3594 S Ocean Blvd #1103...
7,PAST SALE,2024-10-31,Condo/Co-op,701 S Olive Ave #1204,West Palm Beach,FL,33401.0,1225000.0,2.0,2.5,Two City Plaza Condo,1625.0,NaN,2008.0,NaN,754.0,1562.0,Sold,NaN,NaN,https:/

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: 200 East Palmetto Park Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,100,000M to $3,990,000M
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Palm Beach condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Highland Beach, South Palm Beach, Boca Raton, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 114 condo sales totaling $50,169,300 million from Oct. 27th to Nov. 3rd. The previous week, brokers closed 80 condo sales totaling $27,800,000.

Last week’s units sold for an average of $440,082, higher than the $347,880 average price from the previous week. The a

In [48]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Boca-Raton/200-E-Palmetto-Park-Rd-33432/unit-11/home/42664614


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Highland-Beach/3211-S-Ocean-Blvd-33487/unit-802/home/42389287


In [50]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Highland-Beach/4748-S-Ocean-Blvd-33487/unit-1106/home/42397332


In [51]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/West-Palm-Beach/1701-S-Flagler-Dr-33401/unit-1102/home/42519307


In [52]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Boca-Raton/200-SE-Mizner-Blvd-33432/unit-711/home/176042201


In [53]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/West-Palm-Beach/1701-S-Flagler-Dr-33401/unit-1102/home/42519307


In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-28,Condo/Co-op,200 E Palmetto Park Rd Ph -11,Boca Raton,FL,33432.0,3990000.0,3.0,3.5,200 East Palmetto Park Condo,4154.0,180.0,2009.0,NaN,961.0,4167.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Beaches MLS,RX-10918738,N,Y,26.350338,-80.082988,https://www.redfin.com/FL/Boca-Raton/200-E-Pal...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange,200 East Palmetto Park Condo at 200 E Palmetto...
1,PAST SALE,2024-10-28,Condo/Co-op,3211 S Ocean Blvd #802,Highland Beach,FL,33487.0,3050000.0,3.0,4.0,Villa Mare Condo,2657.0,NaN,1991.0,NaN,1148.0,3313.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3211-...,Beaches MLS,RX-11008108,N,Y,26.417668,-80.064013,https://www.redfin.com/FL/Highland-Beach/3211-...,Geoffrey Braboy,Compass Florida LLC,None,None,Geoffrey Braboy,Compass Florida LLC,None,None,2,blue,Villa Mare Condo at 3211 S Ocean Blvd #802 in ...
2,PAST SALE,2024-11-01,Condo/Co-op,200 SE Mizner Blvd #711,Boca Raton,FL,33432.0,2750000.0,3.0,3.0,Alina Boca Raton Condo,2035.0,NaN,2021.0,NaN,1351.0,2663.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-11018049,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Edward Pitlake,Douglas Elliman,Kathy Koch Pitlake,Douglas Elliman,Shaina MacMahon,Compass Florida LLC,None,None,3,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
3,PAST SALE,2024-10-31,Condo/Co-op,3550 S Ocean Blvd Unit 2b,South Palm Beach,FL,33480.0,2625000.0,3.0,3.5,3550 South Ocean Condo,2982.0,NaN,2019.0,NaN,880.0,3558.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Beaches MLS,RX-10863520,N,Y,26.611901,-80.038420,https://www.redfin.com/FL/Palm-Beach/3550-S-Oc...,Patricia Towle,"Sotheby's Intl. Realty, Inc.",None,None,Brie Segota,Coffee Cake And Real Estate,None,None,4,blue,3550 South Ocean Condo at 3550 S Ocean Blvd Un...
4,PAST SALE,2024-10-30,Condo/Co-op,200 SE Mizner Blvd #608,Boca Raton,FL,33432.0,2150000.0,2.0,2.5,Alina Boca Raton Condo,1950.0,381815.0,2021.0,NaN,1103.0,2596.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Beaches MLS,RX-10986420,N,Y,26.347636,-80.083230,https://www.redfin.com/FL/Boca-Raton/200-SE-Mi...,Rochelle LeCavalier,Douglas Elliman,None,None,Rochelle LeCavalier,Douglas Elliman,None,None,5,blue,Alina Boca Raton Condo at 200 SE Mizner Blvd #...
5,PAST SALE,2024-10-31,Condo/Co-op,350 S Ocean Blvd Ph -C,Boca Raton,FL,33432.0,1900000.0,3.0,3.0,Beresford,2296.0,NaN,1977.0,NaN,828.0,3894.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Beaches MLS,RX-10978366,N,Y,26.347273,-80.070212,https://www.redfin.com/FL/Boca-Raton/350-S-Oce...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Mona Pincus,Stonegate Realty Corporation,None,None,6,blue,Beresford at 350 S Ocean Blvd Ph -C in Boca Raton
6,PAST SALE,2024-10-30,Condo/Co-op,3594 S Ocean Blvd #1103,Highland Beach,FL,33487.0,1250000.0,2.0,2.0,Highland Beach Club,1400.0,NaN,1997.0,NaN,893.0,1469.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3594-...,Beaches MLS,F10458939,N,Y,26.411722,-80.065580,https://www.redfin.com/FL/Highland-Beach/3594-...,Tanya Nilsson,The Network Real Estate Broker,None,None,Simon Isaacs,Simon Isaacs Real Estate LLC,None,None,7,blue,Highland Beach Club at 3594 S Ocean Blvd #1103...
7,PAST SALE,2024-10-31,Condo/Co-op,701 S Olive Ave #1204,West Palm Beach,FL,33401.0,1225000.0,2.0,2.5,Two City Plaza Condo,1625.0,NaN,2008.0,NaN,754.0,1562.0,Sold,NaN,NaN,https:/

## Time on Market Calculator

In [57]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 7, 29) ## List (Earlier) date
date2 = datetime(2024, 10, 28) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

91


## Clean CSV for Datawrapper Chart

In [58]:
chart_df = df_top_ten

In [59]:
chart_df = chart_df.fillna(" ")

In [60]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [61]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [62]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [63]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [64]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [65]:
csv_date_string = today.strftime("%m_%d_%Y")

In [66]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [67]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-10-28,200 E Palmetto Park Rd Ph -11 in Boca Raton,"$3,990,000",3,3.5,"4,154",2009,$961,Beaches MLS,RX-10918738,Jonathan Postma Coldwell Banker/BR,Jonathan Postma Coldwell Banker/BR
1,2024-10-28,3211 S Ocean Blvd #802 in Highland Beach,"$3,050,000",3,4.0,"2,657",1991,"$1,148",Beaches MLS,RX-11008108,Geoffrey Braboy Compass Florida LLC,Geoffrey Braboy Compass Florida LLC
2,2024-11-01,200 SE Mizner Blvd #711 in Boca Raton,"$2,750,000",3,3.0,"2,035",2021,"$1,351",Beaches MLS,RX-11018049,Edward Pitlake Douglas Elliman Kathy Koch Pitl...,Shaina MacMahon Compass Florida LLC
3,2024-10-31,3550 S Ocean Blvd Unit 2b in South Palm Beach,"$2,625,000",3,3.5,"2,982",2019,$880,Beaches MLS,RX-10863520,"Patricia Towle Sotheby's Intl. Realty, Inc.",Brie Segota Coffee Cake And Real Estate
4,2024-10-30,200 SE Mizner Blvd #608 in Boca Raton,"$2,150,000",2,2.5,"1,950",2021,"$1,103",Beaches MLS,RX-10986420,Rochelle LeCavalier Douglas Elliman,Rochelle LeCavalier Douglas Elliman
5,2024-10-31,350 S Ocean Blvd Ph -C in Boca Raton,"$1,900,000",3,3.0,"2,296",1977,$828,Beaches MLS,RX-10978366,Gregory Lynn Lynn Realty Group Charli Lynn Lyn...,Mona Pincus Stonegate Realty Corporation
6,2024-10-30,3594 S Ocean Blvd #1103 in Highland Beach,"$1,250,000",2,2.0,"1,400",1997,$893,Beaches MLS,F10458939,Tanya Nilsson The Network Real Estate Broker,Simon Isaacs Simon Isaacs Real Estate LLC
7,2024-10-31,701 S Olive Ave #1204 in West Palm Beach,"$1,225,000",2,2.5,"1,625",2008,$754,Beaches MLS,RX-11033422,Christopher Cronican Douglas Elliman (Palm Bea...,Jenna Steffens Douglas Elliman (Palm Beach)
8,2024-10-31,1701 S Flagler Dr #1102 in West Palm Beach,"$1,100,000",3,2.0,"1,776",1973,$619,Beaches MLS,RX-11012717,Elizabeth Hoadley Purely Palm Beach Real Estat...,Elizabeth Hoadley Purely Palm Beach Real Estat...
9,2024-10-30,4748 S Ocean Blvd #1106 in Highland Beach,"$1,100,000",2,2.0,"1,426",1979,$771,Beaches MLS,RX-11012484,Lisa Bradley Coldwell Banker,Nicole Melveney Douglas Elliman
